In [2]:

val df = Seq(  (1,"Gaga","India","2022-01-11"),
               (1,"Katy","UK","2022-01-11"),
               (1,"Bey","Europe","2022-01-11"),
               (2,"Gaga",null,"2022-10-11"),
               (2,"Katy","India","2022-10-11"),
               (2,"Bey","US","2022-02-15"),
               (3,"Gaga","Europe","2022-10-11"),
               (3,"Katy","US","2022-10-11"),
               (3,"Bey",null,"2022-02-15"), 
               (1,"Gaga","US","2022-01-11"),
               (3,"Katy","Switz","2022-02-15")).toDF("ID","NAME","COUNTRY", "Date_part")

df.show(false)


+---+----+-------+----------+
|ID |NAME|COUNTRY|Date_part |
+---+----+-------+----------+
|1  |Gaga|India  |2022-01-11|
|1  |Katy|UK     |2022-01-11|
|1  |Bey |Europe |2022-01-11|
|2  |Gaga|null   |2022-10-11|
|2  |Katy|India  |2022-10-11|
|2  |Bey |US     |2022-02-15|
|3  |Gaga|Europe |2022-10-11|
|3  |Katy|US     |2022-10-11|
|3  |Bey |null   |2022-02-15|
|1  |Gaga|US     |2022-01-11|
|3  |Katy|Switz  |2022-02-15|
+---+----+-------+----------+



df: org.apache.spark.sql.DataFrame = [ID: int, NAME: string ... 2 more fields]


In [7]:
val pivot_name_df = df.groupBy($"ID", $"Date_part").pivot($"NAME").agg(collect_set($"COUNTRY"))
pivot_name_df.show(false)

+---+----------+--------+-----------+-------+
|ID |Date_part |Bey     |Gaga       |Katy   |
+---+----------+--------+-----------+-------+
|1  |2022-01-11|[Europe]|[India, US]|[UK]   |
|2  |2022-10-11|[]      |[]         |[India]|
|3  |2022-10-11|[]      |[Europe]   |[US]   |
|2  |2022-02-15|[US]    |[]         |[]     |
|3  |2022-02-15|[]      |[]         |[Switz]|
+---+----------+--------+-----------+-------+



pivot_name_df: org.apache.spark.sql.DataFrame = [ID: int, Date_part: string ... 3 more fields]


In [8]:
pivot_name_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- ObjectHashAggregate(keys=[ID#13, Date_part#16], functions=[collect_set(if ((NAME#14 <=> Bey)) COUNTRY#15 else null, 0, 0), collect_set(if ((NAME#14 <=> Gaga)) COUNTRY#15 else null, 0, 0), collect_set(if ((NAME#14 <=> Katy)) COUNTRY#15 else null, 0, 0)])
   +- Exchange hashpartitioning(ID#13, Date_part#16, 200), ENSURE_REQUIREMENTS, [plan_id=388]
      +- ObjectHashAggregate(keys=[ID#13, Date_part#16], functions=[partial_collect_set(if ((NAME#14 <=> Bey)) COUNTRY#15 else null, 0, 0), partial_collect_set(if ((NAME#14 <=> Gaga)) COUNTRY#15 else null, 0, 0), partial_collect_set(if ((NAME#14 <=> Katy)) COUNTRY#15 else null, 0, 0)])
         +- LocalTableScan [ID#13, NAME#14, COUNTRY#15, Date_part#16]




In [21]:
val explode_df = pivot_name_df.withColumn("vars", explode(arrays_zip($"Bey", $"Gaga", $"Katy"))
                        ).select($"ID",$"Date_part",$"vars.Bey", $"vars.Gaga", $"vars.Katy")

explode_df.show(false)
explode_df.explain()

+---+----------+------+------+-----+
|ID |Date_part |Bey   |Gaga  |Katy |
+---+----------+------+------+-----+
|1  |2022-01-11|Europe|India |UK   |
|1  |2022-01-11|null  |US    |null |
|2  |2022-10-11|null  |null  |India|
|3  |2022-10-11|null  |Europe|US   |
|2  |2022-02-15|US    |null  |null |
|3  |2022-02-15|null  |null  |Switz|
+---+----------+------+------+-----+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [ID#13, Date_part#16, vars#719.Bey AS Bey#726, vars#719.Gaga AS Gaga#727, vars#719.Katy AS Katy#728]
   +- Generate explode(arrays_zip(Bey#294, Gaga#296, Katy#298, Bey, Gaga, Katy)), [ID#13, Date_part#16], false, [vars#719]
      +- ObjectHashAggregate(keys=[ID#13, Date_part#16], functions=[collect_set(if ((NAME#14 <=> Bey)) COUNTRY#15 else null, 0, 0), collect_set(if ((NAME#14 <=> Gaga)) COUNTRY#15 else null, 0, 0), collect_set(if ((NAME#14 <=> Katy)) COUNTRY#15 else null, 0, 0)])
         +- Exchange hashpartitioning(ID#13, Date_part#16, 200), ENSURE_REQ

explode_df: org.apache.spark.sql.DataFrame = [ID: int, Date_part: string ... 3 more fields]
